Notebook info

In [ ]:
import tensorflow as tf

#from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras import Model, Sequential

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
tf.__version__

'2.3.0'

In [ ]:
from google.colab import drive
drive.mount()